## 1. Import Data

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('dataset/Customer Churn Banking.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df = df.drop(columns=['RowNumber','CustomerId', 'Surname'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## 2. Deskripsi Data

In [4]:
#Membaca Tipe Kolom Pada Data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


## 3. Cek Duplikasi & Missing Value

In [5]:
#Melihat Jumlah duplikasi baris setiap kolom
df.duplicated().sum()

np.int64(0)

In [6]:
#Melihat Jumlah Baris Yang Kosong
df.isna().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [7]:
#Memisahkan Kolom Numbers dan Kategori
numbers = df.select_dtypes(include = ['number']).columns.drop('Exited')
categories = df.select_dtypes(exclude = ['number']).columns

In [8]:
#Mendeskripsikan Kolom Numerik
round(df.describe(),2)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.00,10000.0
mean,650.53,38.92,5.01,76485.89,1.53,0.71,0.52,100090.24,0.2
std,96.65,10.49,2.89,62397.41,0.58,0.46,0.50,57510.49,0.4
min,350.00,18.00,0.00,0.00,1.00,0.00,0.00,11.58,0.0
25%,584.00,32.00,3.00,0.00,1.00,0.00,0.00,51002.11,0.0
50%,652.00,37.00,5.00,97198.54,1.00,1.00,1.00,100193.92,0.0
75%,718.00,44.00,7.00,127644.24,2.00,1.00,1.00,149388.25,0.0
max,850.00,92.00,10.00,250898.09,4.00,1.00,1.00,199992.48,1.0


In [9]:
#Mendeskripsikan Kolom Kategori
df[categories].describe(include='all')

,Geography,Gender
count,10000,10000
unique,3,2
top,France,Male
freq,5014,5457


## 4. Exploratory Data Analysis

In [10]:
import numpy as np
from scipy.stats import chi2_contingency

In [11]:
def cramers_v(confusion_matrix):
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

In [12]:
results = []

for col in categories:
    # Crosstab: df['Exited'] diasumsikan berisi 0/1
    ct = pd.crosstab(df[col], df['Exited'])

    # Pastikan kolom 0 dan 1 selalu ada
    if 0 not in ct.columns:
        ct[0] = 0
    if 1 not in ct.columns:
        ct[1] = 0

    # Urutkan kolom dan rename supaya enak dibaca
    ct = ct[[0, 1]].rename(columns={0: "No", 1: "Yes"})

    # Chi-square
    chi2, p, _, _ = chi2_contingency(ct)
    p_value = "<0.05" if p < 0.05 else ">0.05"

    # Cramer's V
    coef = cramers_v(ct)

    # Loop setiap kategori
    for value in ct.index:
        no = int(ct.loc[value, "No"])
        yes = int(ct.loc[value, "Yes"])
        total = no + yes
        churn_proba = yes / total if total > 0 else 0

        results.append({
            "column": col,
            "value": value,
            "No": no,
            "Yes": yes,
            "churn_proba": round(churn_proba, 2),
            "p_value": p_value,
            "coef": round(coef, 2),
        })

In [13]:
final_df = pd.DataFrame(results)
final_df.set_index(['column', 'value'], inplace=True)
final_df = final_df.sort_index()
final_df

No   Yes  churn_proba p_value  coef
column    value                                         
Gender    Female   3404  1139         0.25   <0.05  0.11
          Male     4559   898         0.16   <0.05  0.11
Geography France   4204   810         0.16   <0.05  0.17
          Germany  1695   814         0.32   <0.05  0.17
          Spain    2064   413         0.17   <0.05  0.17

In [14]:
#Mengelompokkan usia
df['AgeGroup'] = pd.cut(
    df['Age'],
    bins=[17, 30, 50, 65, 92],
    labels=[
        'Young',
        'Adult',
        'Senior',
        'Elder'
    ]
)

In [15]:
#Membuat kelompok tenure
df['TenureGroup'] = pd.cut(
    df['Tenure'],
    bins=[-1, 3, 6, 10],
    labels=['New', 'Mid', 'Loyal']
)

In [16]:
#Menghitung perbandingan jumlah saldo dan jumlah produk bank yang digunakan pelanggan
df['BalancePerProduct'] = df['Balance'] / (df['NumOfProducts'] + 1)

In [17]:
#Menghitung perbandingan gaji dan jumlah saldo rekening 
df['SalaryBalanceRatio'] = df['EstimatedSalary'] / (df['Balance'] + 1)

In [18]:
#Menghitung engagement score pelanggan
df['EngagementScore'] = (
    df['HasCrCard'] +
    df['IsActiveMember'] +
    (df['NumOfProducts'] > 1).astype(int)
)

In [19]:
#Mengelompokkan credit score
df['CreditScoreGroup'] = pd.cut(
    df['CreditScore'],
    bins=[0, 580, 670, 740, 850],
    labels=['Poor', 'Fair', 'Good', 'Excellent']
)

In [20]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup,TenureGroup,BalancePerProduct,SalaryBalanceRatio,EngagementScore,CreditScoreGroup
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,Adult,New,0.00,101348.880000,2,Fair
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,Adult,New,41903.93,1.342848,1,Fair
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,Adult,Loyal,39915.20,0.713581,2,Poor
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,Adult,New,0.00,93826.630000,1,Good
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,Adult,New,62755.41,0.630093,2,Excellent


### Save To Excel

In [22]:
df.to_excel('CustomerChurnAnalysis.xlsx', index=False)